## Step 1: Load your annotation data
 The original annotations are stored in an Excel file and contain bounding box coordinates and image names.

In [8]:
data_folder = './dataset'
images_folder = f'{data_folder}/images'
annotations = f'{data_folder}/annotations.xlsx'

In [9]:
import pandas as pd

data = pd.read_excel(annotations)
data.head()

,Unnamed: 0,label_name,bbox_x,bbox_y,bbox_width,bbox_height,image_name,image_width,image_height
0,0,sheep,187,68,30,24,DJI_20240125160524_0001_T.JPG,640,512
1,1,sheep,229,29,23,28,DJI_20240125160524_0001_T.JPG,640,512
2,2,sheep,208,21,24,34,DJI_20240125160524_0001_T.JPG,640,512
3,3,sheep,160,1,19,33,DJI_20240125160524_0001_T.JPG,640,512
4,4,sheep,170,10,29,26,DJI_20240125160524_0001_T.JPG,640,512


## Step 2: Prepare the columns for YOLO format
YOLO expects bounding boxes in the format: class_id, x_center, y_center, width, height

We'll rename the columns accordingly and add a class ID column.



In [10]:
dataset = data.loc[:, ['bbox_x', 'bbox_y', 'bbox_width', 'bbox_height', 'image_name']]

dataset.rename(columns={
    'bbox_x': 'x_center',
    'bbox_y': 'y_center',
    'bbox_width': 'width',
    'bbox_height': 'height'
}, inplace=True)

# Since we're only detecting one class (sheep), we can set class_id = 0 for all examples
dataset['class_id'] = 0

dataset.head()

,x_center,y_center,width,height,image_name,class_id
0,187,68,30,24,DJI_20240125160524_0001_T.JPG,0
1,229,29,23,28,DJI_20240125160524_0001_T.JPG,0
2,208,21,24,34,DJI_20240125160524_0001_T.JPG,0
3,160,1,19,33,DJI_20240125160524_0001_T.JPG,0
4,170,10,29,26,DJI_20240125160524_0001_T.JPG,0


## Step 3: Normalize bounding box coordinate

YOLO format requires normalized values between 0 and 1 based on image width and height.
The bounding box should be expressed as a fraction of the image dimensions.

In [11]:
img_width = 640
img_height = 512

# Normalize the coordinates
dataset['x_center'] = dataset['x_center'] / img_width
dataset['y_center'] = dataset['y_center'] / img_height
dataset['width'] = dataset['width'] / img_width
dataset['height'] = dataset['height'] / img_height

# Round to reduce file size and keep consistent formatting
dataset = dataset.round(6)

dataset.head()

,x_center,y_center,width,height,image_name,class_id
0,0.292188,0.132812,0.046875,0.046875,DJI_20240125160524_0001_T.JPG,0
1,0.357812,0.056641,0.035938,0.054688,DJI_20240125160524_0001_T.JPG,0
2,0.325000,0.041016,0.037500,0.066406,DJI_20240125160524_0001_T.JPG,0
3,0.250000,0.001953,0.029688,0.064453,DJI_20240125160524_0001_T.JPG,0
4,0.265625,0.019531,0.045312,0.050781,DJI_20240125160524_0001_T.JPG,0


## Step 4: Split the dataset into train, validation, and test sets

We'll split the dataset at the image level so all annotations for an image remain together.

In [12]:
from sklearn.model_selection import train_test_split

unique_images = dataset['image_name'].unique()

train_imgs, temp_imgs = train_test_split(unique_images, test_size=0.3, random_state=42)
val_imgs, test_imgs = train_test_split(temp_imgs, test_size=0.5, random_state=42)

def get_split(image_name):
    if image_name in train_imgs:
        return 'train'
    elif image_name in val_imgs:
        return 'val'
    else:
        return 'test'

dataset['split'] = dataset['image_name'].apply(get_split)

dataset.head()

,x_center,y_center,width,height,image_name,class_id,split
0,0.292188,0.132812,0.046875,0.046875,DJI_20240125160524_0001_T.JPG,0,train
1,0.357812,0.056641,0.035938,0.054688,DJI_20240125160524_0001_T.JPG,0,train
2,0.325000,0.041016,0.037500,0.066406,DJI_20240125160524_0001_T.JPG,0,train
3,0.250000,0.001953,0.029688,0.064453,DJI_20240125160524_0001_T.JPG,0,train
4,0.265625,0.019531,0.045312,0.050781,DJI_20240125160524_0001_T.JPG,0,train


## Step 5: Export YOLO-format label files

YOLO expects one .txt file per image, each line containing: class_id x_center y_center width height

In [13]:
import shutil
import os

# Create yolo folders
for split in ['train', 'val', 'test']:
    os.makedirs(os.path.join(data_folder, 'labels', split), exist_ok=True)
    os.makedirs(os.path.join(data_folder, 'images', split), exist_ok=True)


for (image_name, split), group in dataset.groupby(['image_name', 'split']):

    # Write labels to file in yolo format
    label_filename = os.path.splitext(image_name)[0] + ".txt"
    label_path = os.path.join(data_folder, 'labels', split, label_filename)
    with open(label_path, 'w') as f:
        for _, row in group.iterrows():
            line = f"{int(row['class_id'])} {row['x_center']} {row['y_center']} {row['width']} {row['height']}\n"
            f.write(line)

    
    # Copy image file
    image_path = os.path.join(data_folder, 'images', image_name)
    if os.path.exists(image_path):
        destination_image_path = os.path.join(data_folder, 'images', split, image_name)
        shutil.copy2(image_path, destination_image_path)
    else:
        print(f"Warning: Image not found - {image_path}")